# Predicting Countries GDP


### Author : Jose Juan Peña Gomez


I was looking for various economic indicators, statistics of countries, I came across UNData Kaggle's source.

The United Nations Statistics Division (UNSD) of the Department of Economic and Social Affairs (DESA) launched a new internet based data service for the global user community. It brings UN statistical databases within easy reach of users through a single entry point (http://data.un.org/). Users can search and download a variety of statistical resources of the UN system.

    https://www.kaggle.com/sudalairajkumar/undata-country-profiles




sS

Please read about the terms of use and policies here.